# IMAGE VARIATION USING STABLE DIFFUSION

<hr></hr>

This is a image to image stable diffusion notebook that takes in an input image and adds some changes to it, depending on some parameters that you can change. Hope you like it!

## **INSTALLING DEPENDENCIES**

In [1]:
!pip install diffusers==0.3.0 transformers ftfy
!pip install  "ipywidgets>=7,<8"
!pip install gradio 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## **IMPORTING DEPENDENCIES**

In [2]:
import gradio as gr
import inspect
import warnings
import numpy as np
from typing import List, Optional, Union
import requests
from io import BytesIO
from PIL import Image
import torch
from torch import autocast
from tqdm.auto import tqdm
from diffusers import StableDiffusionImg2ImgPipeline


## **GENERATE USER ACCESS TOKEN**

## **INITIALIZE AND DOWNLOAD THE MODEL PIPELINE**

In [3]:
device = "cuda"
model_path = "CompVis/stable-diffusion-v1-4"
access_token = "YOUR ACCESS TOKEN FROM HUGGING FACE"

pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_path,
    revision="fp16", 
    torch_dtype=torch.float16,
    use_auth_token=access_token
)
pipe = pipe.to(device)

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

{'trained_betas'} was not found in config. Values will be initialized to default values.


## **DEFINE THE FUNCTION FOR GRADIO INTEGRATION**



*   The `img` parameter is the image, strength parameter is a floating point between 0 to 1. It specifies how much variation we need in our image. The closer strength is to 1, the more changed the input image will become. A rule of thumb is to keep it between 0.2 - 0.5 for best variations.
*   The `seed` parameter is just random. Like if you generated an image and you want to change it totally, jsut change the seed parameter.
*   The `prompt` parameter takes in a text prompt that specifies what change we want in our image. If you do not want any change and just want to add a jittery effect to your original image, leave it blank, and play around with the strength parameter.

<hr></hr>
<br></br>


Now let's see the workflow of the function!

The function first convert the seed parameter to an integer because the model requires seed parameter to be an integer. We then convert the input `img` to a numpy array and resize it to 768 x 512 dimensions. The dimensions are fixed because the model requires the input image to be of the specified dimensions. We then define the model using `torch.Generator` and pass the parameters to the pipeline named `pipe`. We assign it to variable `image`, which contains our changed image. We then return the image.





In [5]:
def generate(img, strength, seed, prompt, width, height):
  height = int(height)
  width = int(width)

  seed =  int(seed)
  img1 = np.asarray(img)
  img2 = Image.fromarray(img1)

  init_image = img2.resize((width, height))

  
  generator = torch.Generator(device=device).manual_seed(seed)
  with autocast("cuda"):
    image = pipe(prompt=prompt, init_image=init_image, strength=strength, guidance_scale=7.5, generator=generator).images[0]

  return image


## **DEFINE GRADIO INTERFACE**

1. Here, we initialize the Gradio interface using `Interface` method, pass in our function `predict`, and specify the inputs to the interface. 
2. The first input is an image drop-down feature for the `img`.
3. The second input is a slider for the `strength` parameter between 0 to 1.
4. The third input is a textbox for the `seed` parameter.
5. The fourth input is also a textbox for the `prompt` parameter. 
6. After  that, we define the outputs for the interface, which is an image using `gr.Image` method of Gradio. 
We then launch the interface!






In [6]:
import gradio as gr

In [ ]:
gr.Interface(
    
    generate,
    title = 'Image to Image using Diffusers',
    inputs=[
        gr.Image(elem_id = "input-image"),
        gr.Slider(0, 1, value=0.05, label ="Strength (keep close to 0 for minimal changes)"),
        gr.Number(label = "Seed"),
        gr.Textbox(label="Prompt (leave blank if you want minimal changes)"),
        gr.Slider(768, 2768, value=64, label ="Width (make sure width is a multiple of 64)"),
        gr.Slider(512, 2512, value=64, label="Height (make sure width is a multiple of 64)")
    ],
    outputs = [
        gr.Image(elem_id="output-image")
        ], css = "#output-image, #input-image, #image-preview {border-radius: 40px !important; background-color : gray !important;} "
).launch(debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://a8500ff89537cf20.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


  0%|          | 0/37 [00:00<?, ?it/s]

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/gradio/routes.py", line 299, in run_predict
    fn_index, raw_input, username, session_state, iterators
  File "/usr/local/lib/python3.7/dist-packages/gradio/blocks.py", line 790, in process_api
    result = await self.call_function(fn_index, inputs, iterator)
  File "/usr/local/lib/python3.7/dist-packages/gradio/blocks.py", line 698, in call_function
    block_fn.fn, *processed_input, limiter=self.limiter
  File "/usr/local/lib/python3.7/dist-packages/anyio/to_thread.py", line 32, in run_sync
    func, *args, cancellable=cancellable, limiter=limiter
  File "/usr/local/lib/python3.7/dist-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.7/dist-packages/anyio/_backends/_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "<ipython-input-5-91244eae66c1>", line 14, in generate
    image = pipe(prompt=pro

## **AAAND THAT'S ALL, THANK YOU FOR LETTING ME PROVIDE MY SERVICES!**